In [1]:
import numpy as np

from pyecom.parsers import HMParser
from pyecom.resources import BaseResource, Generator, Storage

In [2]:
# Load the Excel file

data = HMParser(file_path='data/EC_V4.xlsx', ec_id=1)
data.parse()

In [3]:
print(data.__doc__)


    Inherits from BaseParser.
    Excel reader for the energy community scenario.
    Has as arguments:
    - file_path: str
    - ec_id: int
    - vals: int -> Not used in current version

    When initialized will have the following information:
    - Generators (.generator)
    - Loads (.load)
    - Storage (.storage)
    - Charging stations (.charging_station)
    - Peers (.peers)
    - Vehicles (.vehicle)
    


In [4]:
data.generator

{'p_forecast': array([[0.0, 0.0, 0.0, 0.0, 0.0, 1.012226775, 2.882248633, 6.788123906,
         11.48394025, 15.14569613, 17.5214277, 18.83098385, 18.74890841,
         17.24180717, 14.89047338, 11.25796131, 6.331765741, 2.653460017,
         0.935303552, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 1.518340162, 4.32337295, 10.18218586,
         17.22591038, 22.7185442, 26.28214154, 28.24647578, 28.12336261,
         25.86271075, 22.33571008, 16.88694197, 9.497648612, 3.980190026,
         1.402955327, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.506113387, 1.441124317, 3.394061953,
         5.741970126, 7.572848065, 8.760713848, 9.415491926, 9.374454204,
         8.620903584, 7.445236692, 5.628980657, 3.165882871, 1.326730009,
         0.467651776, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.506113387, 1.441124317, 3.394061953,
         5.741970126, 7.572848065, 8.760713848, 9.415491926, 9.374454204,
         8.620903584, 7.445236692, 

In [5]:
# Define the base import and export values

imports = BaseResource(name='imports',
                       value=np.zeros((1, data.peers['import_contracted_p_max'].shape[1])),
                       lb=np.zeros((1, data.peers['import_contracted_p_max'].shape[1])),
                       ub=np.max(data.peers['import_contracted_p_max'], axis=0),
                       cost=np.max(data.peers['buy_price'], axis=0))

exports = BaseResource(name='exports',
                       value=np.zeros((1, data.peers['export_contracted_p_max'].shape[1])),
                       lb=np.zeros((1, data.peers['export_contracted_p_max'].shape[1])),
                       ub=np.max(data.peers['export_contracted_p_max'], axis=0),
                       cost=np.max(data.peers['sell_price'], axis=0))

In [6]:
# Define the Generators

gen_act = Generator(name='generators',
                    value=np.zeros(data.generator['p_forecast'].shape),
                    lb=np.zeros(data.generator['p_forecast'].shape),
                    ub=data.generator['p_forecast'],
                    cost=data.generator['cost_parameter_b'],
                    renewable=data.generator['type_generator'])

gen_exc_act = Generator(name='exc_generators',
                        value=np.zeros(data.generator['p_forecast'].shape),
                        lb=np.zeros(data.generator['p_forecast'].shape),
                        ub=data.generator['p_forecast'],
                        cost=data.generator['cost_nde'],
                        renewable=data.generator['type_generator'])

In [16]:
# Define the Loads

load_act = BaseResource(name='load_act',
                        value=data.load['p_forecasts'],
                        lb=data.load['p_forecasts'],
                        ub=data.load['p_forecasts'],
                        cost=np.ones(data.load['p_forecasts'].shape))

load_red = BaseResource(name='load_reduce',
                        value=np.zeros(data.load['p_forecasts'].shape),
                        lb=np.zeros(data.load['p_forecasts'].shape),
                        ub=data.load['p_forecasts'],
                        cost=data.load['cost_reduce'])

load_cut = BaseResource(name='load_cut',
                        value=np.zeros(data.load['p_forecasts'].shape),
                        lb=np.zeros(data.load['p_forecasts'].shape),
                        ub=data.load['p_forecasts'],
                        cost=data.load['cost_cut'])

load_ens = BaseResource(name='load_ens',
                        value=np.zeros(data.load['p_forecasts'].shape),
                        lb=np.zeros(data.load['p_forecasts'].shape),
                        ub=data.load['p_forecasts'],
                        cost=data.load['cost_ens'])